### The problem

Consider an airline that has demand for baggage handlers in a domestic airport. THe hourly requrements for 24 hours starting from midnight are 5, 4, 3, 2, 2, 5 6, 7, 9, 11, 13, 14, 11, 9, 11, 12, 14, 15, 18, 20, 16, 10, 7 and 5 respectively. There are 6 shifts for handlers: the first shift may start at midnight, at 1AM, at 2AM or at 3AM. If the first shift started at hour $H$, the next shifts start at $H + 4$, $H + 8$, $H + 12$, $H + 16$ and $H + 20$. Each shift works for 9 consecutive hours. The demand is the same for all days. Using an integer programming model find the minimum number of handlers required that meets the demand and hour at which the first shift should start so that this minimum number of handlers is minimized.

#### Solution

Let us denote number of handlers at $i$-th shift by $x_i$, where $i \in \{ 0, \dots, 5 \}$. Then our problem is to minimize the total number of handlers, i.e.

$$
\min \sum_{i=0}^5 x_i
\text.
$$

We will have four different optimization problems depending on the shift start time. For example, if the shift would start at midnight, then for first eight hours we would have

\begin{array}{}
x_0 + x_5 \ge 5 \\
x_0 + x_5 \ge 4 \\
x_0 + x_5 \ge 3 \\
x_0 + x_5 \ge 2 \\
x_0 + x_2 \ge 2 \\
x_0 + x_2 \ge 5 \\
x_0 + x_2 \ge 6 \\
x_0 + x_2 \ge 7 \\
\qquad\dots
\end{array}

And if the shift would start at 1AM, then the handlers would shift one row down, i.e.

\begin{array}{}
x_4 + x_5 \ge 5 \\
x_0 + x_5 \ge 4 \\
x_0 + x_5 \ge 3 \\
x_0 + x_5 \ge 2 \\
x_0 + x_5 \ge 2 \\
x_0 + x_2 \ge 5 \\
x_0 + x_2 \ge 6 \\
x_0 + x_2 \ge 7 \\
\qquad\dots
\end{array}

Each of those problem can be also presented in a matrix form, for example $A_{12AM}$ would be a $24$ rows by $6$ columns matrix. Where each row would have only two nonzero values corresponding to the current baggage handlers working. 

Then the mimimization problem would be

\begin{array}{}
\min & c^\intercal x \\
\text{subject to} & A_{12AM} x > b \\
\end{array}

where $c \in \mathbf{R}^6$ is a vector of ones and $b \in \mathbf{R}^{24}$ is a vector of corresponding hourly requirements.

We thus have four problems to solve:

\begin{array}{}
\min & c^\intercal x \\
\text{subject to} & A_{12AM} x \ge b \\
\tag{1}
\end{array}

\begin{array}{}
\min & c^\intercal x \\
\text{subject to} & A_{1AM} x \ge b \\
\tag{2}
\end{array}

\begin{array}{}
\min & c^\intercal x \\
\text{subject to} & A_{2AM} x \ge b \\
\tag{3}
\end{array}

\begin{array}{}
\min & c^\intercal x \\
\text{subject to} & A_{3AM} x \ge b \\
\tag{4}
\end{array}

Where only the matrices are different. From the obtained solutions we can manually choose the optimal one. Although the problems are not in the cannonical form (we would have to switch the sign in the inequality), the software we will be using does not require that.

#### Implementation

In [1]:
import numpy as np
from pulp import *

Firstly we need to define the $b$ vector and the $A_i$ matrices

In [2]:
# x vector
SHIFTS = [0, 1, 2, 3, 4, 5]

In [3]:
# b vector
HOURLY_REQUIREMENTS = [5, 4, 3, 2, 2, 5, 6, 7, 9, 11, 13, 14, 11,
                       9, 11, 12, 14, 15, 18, 20, 16, 10, 7, 5]

In [4]:
# A_i matrices
A_12AM = np.array([
    [1, 0, 0, 0, 0, 1],  # 00:00 1st shift starts
    [1, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1],
    [1, 1, 0, 0, 0, 0],  # 04:00 2nd shift starts
    [1, 1, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0],
    [0, 1, 1, 0, 0, 0],  # 08:00 3rd shift starts
    [0, 1, 1, 0, 0, 0],
    [0, 1, 1, 0, 0, 0],
    [0, 1, 1, 0, 0, 0],
    [0, 0, 1, 1, 0, 0],  # 12:00 4th shift starts
    [0, 0, 1, 1, 0, 0],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 0, 1, 1, 0],  # 16:00 5th shift starts
    [0, 0, 0, 1, 1, 0],
    [0, 0, 0, 1, 1, 0],
    [0, 0, 0, 1, 1, 0],
    [0, 0, 0, 0, 1, 1],  # 20:00 6th shift starts
    [0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 1, 1],
])

A_1AM = np.array([
    [0, 0, 0, 0, 1, 1],
    [1, 0, 0, 0, 0, 1],  # 01:00 1st shift starts
    [1, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1],
    [1, 1, 0, 0, 0, 0],  # 05:00 2nd shift starts
    [1, 1, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0],
    [0, 1, 1, 0, 0, 0],  # 09:00 3rd shift starts
    [0, 1, 1, 0, 0, 0],
    [0, 1, 1, 0, 0, 0],
    [0, 1, 1, 0, 0, 0],
    [0, 0, 1, 1, 0, 0],  # 13:00 4th shift starts
    [0, 0, 1, 1, 0, 0],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 0, 1, 1, 0],  # 17:00 5th shift starts
    [0, 0, 0, 1, 1, 0],
    [0, 0, 0, 1, 1, 0],
    [0, 0, 0, 1, 1, 0],
    [0, 0, 0, 0, 1, 1],  # 21:00 6th shift starts
    [0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 1, 1],
])

A_2AM = np.array([
    [0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 1, 1],
    [1, 0, 0, 0, 0, 1],  # 02:00 1st shift starts
    [1, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1],
    [1, 1, 0, 0, 0, 0],  # 06:00 2nd shift starts
    [1, 1, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0],
    [0, 1, 1, 0, 0, 0],  # 10:00 3rd shift starts
    [0, 1, 1, 0, 0, 0],
    [0, 1, 1, 0, 0, 0],
    [0, 1, 1, 0, 0, 0],
    [0, 0, 1, 1, 0, 0],  # 14:00 4th shift starts
    [0, 0, 1, 1, 0, 0],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 0, 1, 1, 0],  # 18:00 5th shift starts
    [0, 0, 0, 1, 1, 0],
    [0, 0, 0, 1, 1, 0],
    [0, 0, 0, 1, 1, 0],
    [0, 0, 0, 0, 1, 1],  # 22:00 6th shift starts
    [0, 0, 0, 0, 1, 1],
])

A_3AM = np.array([
    [0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 1, 1],
    [1, 0, 0, 0, 0, 1],  # 03:00 1st shift starts
    [1, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1],
    [1, 1, 0, 0, 0, 0],  # 07:00 2nd shift starts
    [1, 1, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0],
    [0, 1, 1, 0, 0, 0],  # 11:00 3rd shift starts
    [0, 1, 1, 0, 0, 0],
    [0, 1, 1, 0, 0, 0],
    [0, 1, 1, 0, 0, 0],
    [0, 0, 1, 1, 0, 0],  # 15:00 4th shift starts
    [0, 0, 1, 1, 0, 0],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 0, 1, 1, 0],  # 19:00 5th shift starts
    [0, 0, 0, 1, 1, 0],
    [0, 0, 0, 1, 1, 0],
    [0, 0, 0, 1, 1, 0],
    [0, 0, 0, 0, 1, 1],  # 23:00 6th shift starts
])

Then for each matrix we will define a problem using python `pulp` libary, solve it and add results to solutions list.

In [5]:
solutions = []

In [6]:
for A in [A_12AM, A_1AM, A_2AM, A_3AM]:
    # Creating the LP problem.
    problem = LpProblem("", LpMinimize)
    # Creating a representation of problem variables
    var_names = LpVariable.dict("Shift", SHIFTS, 0, None, LpInteger)
    # Adding the objective function.
    problem += lpSum(var_names), "Total number of handlers"
    # Adding the constraints.
    for i, (row, hours) in enumerate(zip(A, HOURLY_REQUIREMENTS)):
        problem += lpSum([row[i] * var_names[i] for i in var_names]) >= hours, f"Hour{i:02}:00"
    # Solving the problem.
    problem.solve();
    if not problem.status == 1:
        raise Exception("Unable to solve the problem.");
    solution_location = tuple(_.varValue for _ in problem.variables())
    solutions.append((solution_location, np.sum(solution_location)))    

In [7]:
for solution, value in solutions:
    print(f"Solution at {solution} with value {value}.")

Solution at (0.0, 11.0, 3.0, 9.0, 11.0, 5.0) with value 39.0.
Solution at (0.0, 14.0, 0.0, 14.0, 6.0, 4.0) with value 38.0.
Solution at (0.0, 14.0, 0.0, 18.0, 2.0, 5.0) with value 39.0.
Solution at (0.0, 14.0, 0.0, 20.0, 0.0, 6.0) with value 40.0.


If we look at the fourth solutions we will see that the minimum value of $38$ workers is obtained at point $[ 0, 14, 0, 14, 6, 4]$ so we dont ever need the 1st nor 3rd shift, which might be suprising!

What is even more interesting is that the solution for shifts starting at 3AM. For only 2 more baggage handlers, we can have only 3 shifts, which could, maybe, lower the management costs, although we did not have that constraint in the problem.